# OSRS Pricing Tooling

 - [x] Get OSRS Mapping Data
 - [x] Get OSRS Timeseries Data
 - [x] Identify buy/no-buy and profit for battlestaff & bow creating -> high-alching
 - [ ] Enhanced buy/no-buy logic
 - [ ] Generate plots... maybe via R's ggplot2
 - [ ] Identify high-alch profits
 - [ ] Generate HTML using markdown
 - [ ] Publish to GitLab Pages
 - [ ] Turn into Python script
 - [ ] Automate

In [187]:
import polars as pl
import pandas as pd

## Query OSRS Mapping API

 - Pull data for every item ID
 - Normalize into Pandas DF
 - Convert Pandas DF to Polars DF
 - Declare column data types

In [188]:
import requests

url = 'https://prices.runescape.wiki/api/v1/osrs/mapping'

headers = {
    'User-Agent': 'Major Discount Notification 0.1',
    'From': 'j.grant.redskins@gmail.com'
}

# Polars native functions not working...
# response = requests.get(url, headers=headers)
# df_map = pl.from_dict(response.json())
# df_map = pl.read_json(response.json())
df_map = pd.json_normalize(requests.get(url, headers=headers).json())
df_map = pl.DataFrame(df_map).with_columns(
    [
        pl.col("examine").cast(pl.Utf8), pl.col("id").cast(pl.Int64), pl.col("members").cast(pl.Boolean),
        pl.col("lowalch").cast(pl.Float64), pl.col("limit").cast(pl.Float64), pl.col("value").cast(pl.Int64),
        pl.col("highalch").cast(pl.Float64), pl.col("icon").cast(pl.Utf8), pl.col("name").cast(pl.Utf8),
    ]
)
df_map.sample(3)

## Query OSRS Timeseries API

 - Identify Item IDs to query
  - Only allowed to query a single ID per request
  - Unneeded for output items being high-alched
   - *but* query for items to be purchased solely for high alching profit
 - Pull data for every item ID
   - Normalize into Pandas DF
   - Convert Pandas DF to Polars DF
   - Declare column data types

In [194]:
osrs_item_ids = [
    563, 562, 560, 565, 573, 569, 571, 1777, 5295, 257, 207, 99, 231, 139, 561, 575, 569, 571, 1515
]
osrs_high_alch_item_ids = [
    1397, 1399, 1393, 1395, 855
]
# df_map.filter(pl.col('id').is_in((osrs_item_ids + osrs_high_alch_item_ids)))#.sample(3)

base_url = 'https://prices.runescape.wiki/api/v1/osrs/timeseries?timestep=1h&id='
headers = {
    'User-Agent': 'Major Discount Notification 0.1',
    'From': 'j.grant.redskins@gmail.com'  # This is another valid field
}

df_timeseries = pl.DataFrame()

# Query for each itemID in array/list, then declare data types
for item_id in osrs_item_ids:
    # print(base_url + str(item_id))
    response = requests.get(
        base_url + str(item_id), headers=headers
    )
    df_timeseries = pl.concat(
        [
            df_timeseries, pl.DataFrame(
                pd.json_normalize(response.json(), record_path=['data'])
            ).lazy().with_column(
                pl.lit(item_id).alias('id')
            ).with_columns(
                [
                    pl.col("timestamp").cast(pl.Int64), pl.col("avgHighPrice").cast(pl.Int64), pl.col("avgLowPrice").cast(pl.Int64),
                    pl.col("highPriceVolume").cast(pl.Int64), pl.col("lowPriceVolume").cast(pl.Int64), pl.col("id").cast(pl.Int64)
                ]
            ).with_column(
                (pl.col("timestamp") * 1e3).cast(pl.Datetime).dt.with_time_unit("ms").alias("datetime")
            ).with_column(
                pl.col("datetime").dt.strftime(fmt="%Y-%m-%d %H").cast(pl.Utf8).alias("ymd_h")
            ).collect()
        ], how = "diagonal", rechunk = True #vertical
    )

In [195]:
df_timeseries.sample(3)

timestamp,avgHighPrice,avgLowPrice,highPriceVolume,lowPriceVolume,id,datetime,ymd_h
i64,i64,i64,i64,i64,i64,datetime[ms],str
1665183600,208,209,37361,106278,1515,2022-10-07 23:00:00,"""2022-10-07 23"""
1664629200,7640,7579,18340,6593,99,2022-10-01 13:00:00,"""2022-10-01 13"""
1664751600,1698,1673,1018,561,571,2022-10-02 23:00:00,"""2022-10-02 23"""


### Write Out Timeseries Data

**Timeseries API RESTful service**: 
"Gives a list of the high and low prices of item with the given id at the given interval, up to 365 maximum."

If we want to do long-term trending, more than the 365 maximum, we need to incrementally grow the data by appending data written to "disk"

 - Check if data (parquet file) already exists
  - If exists:
    - read in missing (historical data not included in data pull limitations, 365 1h records)
 - Write data out to parquet file, zstd compression

In [207]:
from os.path import exists
from pathlib import Path

data_directory = "data/"
Path(data_directory).mkdir(parents=True, exist_ok=True)

parquest_path = data_directory + "osrs_timeseries_df.parquet"
parquet_exists = exists(parquest_path)

# if data (parquet) already exists, then add missing (historical) data to the data set
if parquet_exists:
    df_timeseries = pl.concat(
        [
            df_timeseries,
            # historical records saved off from previous run
            pl.scan_parquet(parquest_path).filter(
                pl.col("datetime") < df_timeseries.select("datetime").min()[0,0]
            ).collect()
        ], how = "vertical", rechunk = True
    )

# either write new (if file does not exist) or overwrite file with historical + new data
df_timeseries.write_parquet(parquest_path, compression = "zstd")

does not exist


## Join Map & Timeseries Dataframes, Calculate Median & Most Recent Prices

In [200]:
df_results = df_timeseries.lazy().groupby("id").agg(
    pl.col("avgHighPrice").median()
).rename({"avgHighPrice": "med_avgHighPrice"}).join(
    df_timeseries.lazy().groupby("id").agg(
        [
            pl.all().sort_by('datetime').last()
        ]
    ).rename(
        {"avgHighPrice": "last_avgHighPrice", "avgLowPrice": "last_avgLowPrice"}
    ).select(["id", "last_avgHighPrice", "last_avgLowPrice"]),
    on="id", how="inner"
).join(
    df_map.lazy().filter(pl.col('id').is_in((osrs_item_ids + osrs_high_alch_item_ids))).select(
        ["id", "highalch", "members", "name"]
    ),
    on="id", how="inner"
).collect()

df_results.sample(2)

id,med_avgHighPrice,last_avgHighPrice,last_avgLowPrice,highalch,members,name
i64,f64,i64,i64,f64,bool,str
561,139.0,136,134,108.0,false,"""Nature rune"""
562,79.0,79,77,54.0,false,"""Chaos rune"""


### Helper functions for generating buy/no-buy markdown

In [202]:
def gen_staff_markdown(df_map, df_results, x_battlestaff_id, orb_id, nature_id = 561, battlestaff_price = 7000):

    df_orb = df_results.lazy().filter(pl.col("id") == orb_id).with_columns(
        [
            (
                (
                    # X battlestaff high alch price - (nature rune price + battlestaff_price)
                    df_map.filter(pl.col("id") == x_battlestaff_id).select("highalch")[0,0] - 
                    (df_results.filter(pl.col("id") == nature_id).select("last_avgHighPrice")[0,0] + battlestaff_price)
                ) - pl.col("last_avgHighPrice")
            ).alias("profit"),
            (
                pl.col("last_avgHighPrice")/pl.col("med_avgHighPrice")
            ).round(2).alias("discount")
        ]
    ).with_columns(
        [
            pl.when(
                    pl.col("profit") >= 0
                ).then("Buy").otherwise("Don't Buy").alias("if_Buy_str"),
            pl.when(pl.col("profit") >= 0).then(
                pl.concat_str(
                    [
                        pl.lit(" between"),
                        pl.col("last_avgLowPrice"),
                        pl.lit("and"),
                        pl.col("last_avgHighPrice")
                    ], sep = " "
                )
            ).otherwise("").alias("extra_str")
        ]
    ).collect()

    orb = (
        "### " + df_orb.select("name")[0,0] + ":\n<br /> **" + str(df_orb.select("if_Buy_str")[0,0]) + "**" +
        str(df_orb.select("extra_str")[0,0]) + "\n<br />" +
        "**Profit**: *" + str(df_orb.select("profit")[0,0]) + "*" +
        "**Value**: *" + str(df_orb.select("discount")[0,0]) + "x against historical median*"
    )

    return(orb)

def gen_bow_markdown(df_map, df_results, x_bow_id, log_id, nature_id = 561, bowstring_id = 1777):

    df_bowstring = df_results.lazy().filter(pl.col("id") == bowstring_id).with_columns(
        [
            (
                (
                    # X bow high alch price - (nature rune price + bowstring price + log price)
                    df_map.filter(pl.col("id") == x_bow_id).select("highalch")[0,0] - 
                    (
                        df_results.filter(pl.col("id") == nature_id).select("last_avgHighPrice")[0,0] + 
                        # df_results.filter(pl.col("id") == bowstring_id).select("last_avgHighPrice")[0,0] +
                        df_results.filter(pl.col("id") == log_id).select("last_avgHighPrice")[0,0]
                    )
                ) - pl.col("last_avgHighPrice")
            ).alias("profit"),
            (
                pl.col("last_avgHighPrice")/pl.col("med_avgHighPrice")
            ).round(2).alias("discount")
        ]
    ).with_columns(
        [
            pl.when(
                    pl.col("profit") >= 0
                ).then("Buy").otherwise("Don't Buy").alias("if_Buy_str"),
            pl.when(pl.col("profit") >= 0).then(
                pl.concat_str(
                    [
                        pl.lit(" between"),
                        pl.col("last_avgLowPrice"),
                        pl.lit("and"),
                        pl.col("last_avgHighPrice")
                    ], sep = " "
                )
            ).otherwise("").alias("extra_str")
        ]
    ).collect()

    bow_string = (
        "### " + df_bowstring.select("name")[0,0] + ":\n<br /> **" + str(df_bowstring.select("if_Buy_str")[0,0]) + "**" +
        str(df_bowstring.select("extra_str")[0,0]) + "\n<br />" +
        "**Profit**: *" + str(df_bowstring.select("profit")[0,0]) + "*" +
        "**Value**: *" + str(df_bowstring.select("discount")[0,0]) + "x against historical median*"
    )

    return(bow_string)

In [203]:
# gen_staff_markdown(df_map, df_results, x_battlestaff_id = 1397, orb_id = 573)
# gen_staff_markdown(df_map, df_results, x_battlestaff_id = 1399, orb_id = 575)
# gen_staff_markdown(df_map, df_results, x_battlestaff_id = 1393, orb_id = 569)
# gen_staff_markdown(df_map, df_results, x_battlestaff_id = 1395, orb_id = 571)

gen_bow_markdown(df_map, df_results, x_bow_id = 855, log_id = 1515, nature_id = 561, bowstring_id = 1777)

'### Bow string:\n<br /> **Buy** between 76 and 77\n<br />**Profit**: *350.0***Value**: *1.0x against historical median*'